In [7]:
from PIL import Image, ImageTk
import tkinter as tk
import cv2
import os
import numpy as np
from tensorflow.keras.models import model_from_json
import operator
import time
import sys, os
import matplotlib.pyplot as plt
import hunspell
from string import ascii_uppercase
import speech_recognition as sr
import pyttsx3 

mixed_list=['J','K','L','M','N','O','P','Q','R','S','A','T','U','V','W','X','Y','Z','HELLO','MALE','FEMALE','B','YES','NO','WHERE','WHEN','WHAT','EMAIL','NAME','HOUSE','WORK','TIME','C','HELP','I LOVE YOU','STAND','FAMILY','MARRIAGE','YOU','WE','NICE','MEET','OWE','D','COME','GO','PAINT','E','F','G','H','I']
alphabet=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

class Application:
    def __init__(self):
        self.flag=0
        self.hs = hunspell.Hunspell("C:/Users/a/Desktop/BTech/Project/Docs/en_US", "C:/Users/a/Desktop/BTech/Project/Docs/en_US.dic")
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
         
        self.speak_engine=pyttsx3.init()
        self.speak_engine.setProperty("rate",100)
        voices=self.speak_engine.getProperty("voices")
        self.speak_engine.setProperty("voice",voices[0].id)   
             
        self.json_file = open("C:/Users/a/Downloads/MODEL_WF-mixed_V4.json", "r")
        self.model_json1 = self.json_file.read()
        self.json_file.close()
        self.loaded_model = model_from_json(self.model_json1)       
        self.loaded_model.load_weights("C:/Users/a/Downloads/MODEL_WF-mixed_V4.h5")
        
        self.ct = {}
        self.ct1 = {}
        self.ct['blank'] = 0
        self.blank_flag = 0
        for i in mixed_list:
            self.ct[i] = 0
        print("Loaded model from disk")
        self.root = tk.Tk()
        self.root.title("TWO-WAY COMMUNICATION IN ASL")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x1300")
        self.panel = tk.Label(self.root)
        self.panel.place(x = -60, y = 50, width = 800, height = 400)   #panel big box image
        self.panel2 = tk.Label(self.root) # initialize image panel
        self.panel2.place(x = 350, y = 50, width = 310, height = 310) #panel 2 small gray box image
        
        self.T = tk.Label(self.root)
        self.T.place(x=250,y = 2)
        self.T.config(text = "TWO-WAY COMMUNICATION IN ASL",fg="purple",font=("Times New Roman",26,"bold"))
        
        self.panel3 = tk.Label(self.root) # Current SYmbol
        self.panel3.place(x = 300,y=470)                                         
        self.T1 = tk.Label(self.root)
        self.T1.place(x = 10,y = 470)                                            
        self.T1.config(text="Character :",font=("Courier",20,"bold"))
        
        self.panel4 = tk.Label(self.root) # Word
        self.panel4.place(x = 165,y=510)                  
        self.T2 = tk.Label(self.root)
        self.T2.place(x = 10,y = 510)                     
        self.T2.config(text ="Word :",font=("Courier",20,"bold"))
        
        
        self.panel5 = tk.Label(self.root) # Sentence
        self.panel5.place(x = 180,y=550)       
        self.T3 = tk.Label(self.root)
        self.T3.place(x = 10,y =550)           
        self.T3.config(text ="Sentence :",font=("Courier",20,"bold"))
      
        self.T3_1 = tk.Label(self.root)
        self.T3_1.place(x = 700,y =80)           
        self.T3_1.config(text ="person 1 :",font=("Courier",20,"bold"))
               
        self.T3_2 = tk.Label(self.root)
        self.T3_2.place(x = 1125,y =210)           
        self.T3_2.config(text ="person 2", font=("Courier",20,"bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x = 250,y =580)  
        self.T4.config(text = "Suggestions",fg="red",font = ("Courier",20,"bold"))
        
        self.btcall = tk.Button(self.root,command = self.action_call,height = 0,width = 0)
        self.btcall.config(text = "About",font = ("Courier",14))
        self.btcall.place(x = 1200, y = 0)
        
        self.btcall2 = tk.Button(self.root,command = self.action_call2,height = 0,width = 0)
        self.btcall2.config(text = "Guide",font = ("Courier",14))
        self.btcall2.place(x = 1110, y = 0)

     
        self.bt1=tk.Button(self.root, command=self.action1,height = 0,width = 0) 
        self.bt1.place(x = 25,y=650)  
        
        self.bt2=tk.Button(self.root, command=self.action2,height = 0,width = 0) 
        self.bt2.place(x = 150,y=650)
        
        self.bt3=tk.Button(self.root, command=self.action3,height = 0,width = 0) 
        self.bt3.place(x = 300,y=650)
        
        self.bt4=tk.Button(self.root, command=self.action4,height = 0,width = 0)
        self.bt4.place(x = 450,y=650)

        self.bt5=tk.Button(self.root, command=self.action5,height = 0,width = 0)
        self.bt5.place(x = 625,y=650)
        
        self.bt3_2=tk.Button(self.root, height = 5,width = 20, wraplength=150)   #user - listen
        self.bt3_2.place(x = 725,y=215)
        
        self.bt7=tk.Button(self.root, text="Start", command=self.start,font=("Courier",16))
        self.bt7.place(x = 575,y=450)
        
        self.bt8=tk.Button(self.root, text="Stop ", command=self.stop,font=("Courier",16))
        self.bt8.place(x = 575,y=495)
        
        self.clear = tk.Button(self.root)
        self.clear.place(x=1060, y=450)
        self.clear.config(text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        
        self.listen = tk.Button(self.root)
        self.listen.place(x=900, y=450)
        self.listen.config(text="Speak", font=("Courier", 20), wraplength=100, command=self.listen_fun)
        
        self.speak = tk.Button(self.root)
        self.speak.place(x=760, y=450)
        self.speak.config(text="Listen", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.str = " "
        self.ccc=0
        self.word = " "
        self.current_symbol = ""
        self.current_symbol1= ""
        self.photo = "Empty"

        self.word1= " "
        self.word2 = " "
        self.word3 = " "
        self.word4 = " "

        self.video_loop()

    def video_loop(self):
        ok, frame = self.vs.read()
        if ok:
            cv2image = cv2.flip(frame, 1)
            x1 = int(0.5*frame.shape[1])
            y1 = 10
            x2 = frame.shape[1]-10
            y2 = int(0.5*frame.shape[1])
            cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)
            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
            cv2image = cv2image[y1:y2, x1:x2]
            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray,(5,5),2)
            th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            self.predict(res)
            self.current_image2 = Image.fromarray(res)
            imgtk = ImageTk.PhotoImage(image=self.current_image2)
            
            self.panel2.imgtk = imgtk
            self.panel2.config(image=imgtk)
            self.panel3.config(text=self.current_symbol1,font=("Courier",20))
            
            self.panel3.config(text=self.current_symbol,font=("Courier",20))
            self.panel4.config(text=self.word,font=("Courier",20))
            self.panel5.config(text=self.str,font=("Courier",20))
            
            self.bt3_1=tk.Button(self.root, height = 2,width = 20)  #person 1 
            self.bt3_1.place(x = 850,y=80)
            self.bt3_1.config(text=self.str, font=("Courier", 20), wraplength=500)
            
            if self.current_symbol in alphabet:
                predicts=self.hs.suggest(self.word)
                if(len(predicts) > 0):
                    self.bt1.config(text=predicts[0],font = ("Courier",20))
                else:
                    self.bt1.config(text="")

#                 if(len(predicts) > 1):
#                     self.bt2.config(text=predicts[1],font = ("Courier",20))
#                 else:
#                     self.bt2.config(text="")
                self.bt2.config(text=self.word,font = ("Courier",20))

                if(len(predicts) > 2):
                    self.bt3.config(text=predicts[2],font = ("Courier",20))
                else:
                    self.bt3.config(text="")

                if(len(predicts) > 3):
                    self.bt4.config(text=predicts[3],font = ("Courier",20))
                else:
                    self.bt4.config(text="")

                if(len(predicts) > 4):
                    self.bt5.config(text=predicts[4],font = ("Courier",20))
                else:
                    self.bt5.config(text="")
            else:
                self.bt1.config(text=self.current_symbol,font = ("Courier",20))                
        self.root.after(30, self.video_loop)  #30 milisecs for real time 
        
        
    def predict(self,test_image):
        if self.flag==1:
            test_image = cv2.resize(test_image, (128,128))
           
            result = self.loaded_model.predict(test_image.reshape(1, 128, 128, 1))       
            prediction={}
            prediction['blank'] = result[0][0]
            inde = 1
            for i in mixed_list:
                prediction[i] = result[0][inde]
                inde += 1
        #LAYER 1
            prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
            self.current_symbol = prediction[0][0]


        #This code checks if the predicted symbol is 'blank', which indicates that no gesture was detected. 
        #If this is the case, it sets the count for each letter in ascii_uppercase to 0. 
        #If the predicted symbol is not 'blank', it increments the count for that symbol in the ct dictionary. 
        #The ct dictionary keeps track of the count of each symbol that has been predicted so far.
            if(self.current_symbol == 'blank'):
                for i in mixed_list:
                    self.ct[i] = 0             
            self.ct[self.current_symbol] += 1

            if(self.ct[self.current_symbol] > 30): 
                for i in mixed_list:
                    if i == self.current_symbol:
                        continue
                    tmp = self.ct[self.current_symbol] - self.ct[i]
                    if tmp < 0:
                        tmp *= -1
                    if tmp <= 20:
                        self.ct['blank'] = 0
                        for i in mixed_list:
                            self.ct[i] = 0
                        return
                self.ct['blank'] = 0
                for i in mixed_list:
                    self.ct[i] = 0
                if self.current_symbol in alphabet:
                    if self.current_symbol == 'blank':
                        if self.blank_flag == 0:
                            self.blank_flag = 1
                            if len(self.str) > 0:
                                self.str += " "
                            self.str += self.word
                            self.word = ""
                    else:
                        if(len(self.str) > 16):       
                            self.str = ""
                        self.blank_flag = 0
                        self.word += self.current_symbol
                else:
                    self.word=self.current_symbol
                    if self.current_symbol == 'blank':
                        if self.blank_flag == 0:
                            self.blank_flag = 1
                            if len(self.str) > 0:
                                self.str += " "
                            self.str += self.word
                            self.word = ""
    def action1(self):
        if self.current_symbol in alphabet:
            predicts=self.hs.suggest(self.word)
            print(predicts)
            if(len(predicts) > 0):
                self.word=""
                self.str+=" "
                self.str+=predicts[0]
        else:
            self.str+=self.current_symbol+" "
    def action2(self):
#     	predicts=self.hs.suggest(self.word)
#     	if(len(predicts) > 1):
        self.str+=self.word
    def action3(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 2):
            self.word=""
            self.str+=" "
            self.str+=predicts[2]
    def action4(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 3):
            self.word=""
            self.str+=" "
            self.str+=predicts[3]
    def action5(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 4):
            self.word=""
            self.str+=" "
            self.str+=predicts[4]    
    
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
    
    def destructor1(self):
        print("Closing Application...")
        self.root1.destroy()
        
    def destructor2(self):
        print("Closing Application...")
        self.root2.destroy()
        
    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()
        
    def start(self):
        self.flag=1
    
    def stop(self):
        self.flag=0
        self.current_symbol=""
        self.word=""
        
    def clear_fun(self):
        self.str=" "
        self.word=" "
        self.word1 = " "
        self.word2 = " "
        self.word3 = " "
        self.word4 = " "
        self.bt3_2.config(text="")
        
    def listen_fun(self):
        r = sr.Recognizer()
        
        with sr.Microphone() as source:    #since we are taking online audio
            r.adjust_for_ambient_noise(source)  #to adjust ambient noise
            audio = r.listen(source)
            
            try:
                dest = r.recognize_google(audio)
                self.bt3_2.config(text= dest,font = ("Courier",20))
            except Exception as e:
                print("Error : " + str(e))
                
    def action_call2(self) :
        
        self.root2 = tk.Toplevel(self.root)
        self.root2.title("Guide")
        self.root2.protocol('WM_DELETE_WINDOW', self.destructor2)
        self.root2.geometry("1000x1000") 
        
        self.tx2_1 = tk.Label(self.root2)
        self.tx2_1.place(x = 300,y = 20)
        self.tx2_1.config(text = "Step By Step Process", fg="red", font = ("Courier",30,"bold"))
        
        self.tx2_2 = tk.Label(self.root2)
        self.tx2_2.place(x = 25,y = 200)
        self.tx2_2.config(text = "1.Click the start button.", font = ("Courier",15,"bold"))
        
        self.tx2_3 = tk.Label(self.root2)
        self.tx2_3.place(x = 25,y = 250)
        self.tx2_3.config(text = "2.Perform hand gestures. Make sure it is visible in panel 2 (Gaussian blur filter window)", font = ("Courier",15,"bold"))
        
        self.tx2_4 = tk.Label(self.root2)
        self.tx2_4.place(x = 25,y = 300)
        self.tx2_4.config(text = "3.Click the Stop button once the sentence is formed.", font = ("Courier",15,"bold"))
        
        self.tx2_5 = tk.Label(self.root2)
        self.tx2_5.place(x = 25,y = 350)
        self.tx2_5.config(text = "4.Press the Listen button to listen to the sentence.", font = ("Courier",15,"bold"))
        
        self.tx2_6 = tk.Label(self.root2)
        self.tx2_6.place(x = 25,y = 400)
        self.tx2_6.config(text = "5.Press the Speak button to communicate with the signer person which will get converted to text.", font = ("Courier",15,"bold"))
        
        
        self.tx2_7 = tk.Label(self.root2)
        self.tx2_7.place(x = 25,y = 450)
        self.tx2_7.config(text = "6.Clear everything by pressing the Clear button.", font = ("Courier",15,"bold"))
       
        self.tx2_8 = tk.Label(self.root2)
        self.tx2_8.place(x = 25,y = 500)
        self.tx2_8.config(text = "7.Repeat step 1.", font = ("Courier",15,"bold"))



        

    def action_call(self) :
        
        self.root1 = tk.Toplevel(self.root)
        self.root1.title("About")
        self.root1.protocol('WM_DELETE_WINDOW', self.destructor1)
        self.root1.geometry("800x800") 
        
        self.tx = tk.Label(self.root1)
        self.tx.place(x = 330,y = 20)
        self.tx.config(text = "Efforts By", fg="red", font = ("Courier",30,"bold"))

#         self.photo1 = tk.PhotoImage(file="C:/Users/Admin/Downloads/chinmay.png")
        self.w1 = tk.Label(self.root1, image = self.photo1)
        self.w1.place(x = 85, y = 100)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 75,y = 310)
        self.tx6.config(text = "Chinmay Bhat \n 111907013", font = ("Courier",15,"bold"))

#         self.photo2 = tk.PhotoImage(file='C:/Users/Admin/Downloads/ruts_1.png')
        self.w2 = tk.Label(self.root1, image = self.photo2)
        self.w2.place(x = 445, y = 100)
        self.tx2 = tk.Label(self.root1)
        self.tx2.place(x = 405,y = 310)
        self.tx2.config(text = "Rutuja Rajeshirke \n 111907042", font = ("Courier",15,"bold"))

#         self.photo4 = tk.PhotoImage(file="C:/Users/Admin/Downloads/sanss.png")
        self.w4 = tk.Label(self.root1, image = self.photo4)
        self.w4.place(x = 85, y = 370)
        self.tx4 = tk.Label(self.root1)
        self.tx4.place(x = 80,y = 590)
        self.tx4.config(text = "Sanskruti Chude \n 111907045", font = ("Courier",15,"bold"))
        
#         self.photo5 = tk.PhotoImage(file="C:/Users/Admin/Downloads/vaibhaviiii.png")
        self.w5 = tk.Label(self.root1, image = self.photo5)
        self.w5.place(x = 445, y = 370)
        self.tx5 = tk.Label(self.root1)
        self.tx5.place(x = 440,y = 590)
        self.tx5.config(text = "Vaibhavi Mhaiskar \n 111907059", font = ("Courier",15,"bold"))
        
print("Starting Application...")
pba = Application()
pba.root.mainloop()

Starting Application...
Loaded model from disk
Closing Application...


Exception in Tkinter callback
Traceback (most recent call last):
  File "F:\Anacondanav\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\a\AppData\Local\Temp\ipykernel_9752\2773439996.py", line 413, in action_call
    self.w1 = tk.Label(self.root1, image = self.photo1)
AttributeError: 'Application' object has no attribute 'photo1'


Closing Application...
Closing Application...
